In [2]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import t
from scipy import stats
u = np.random.normal(0, 1, 100)
X = np.random.uniform(0, 1, (100, 3))
one = np.ones(100)
beta = np.array([1,2,0])
y = one + np.dot(X,beta) + u
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
beta_hat = results.params
y_hat = np.dot(X, beta_hat)
cov = np.linalg.inv(np.dot(X.transpose(),X))
for i in range(4):
    left = beta_hat[i] - t.ppf(0.975, 96) * np.sqrt(cov[i,i])
    right = beta_hat[i] + t.ppf(0.975,96) * np.sqrt(cov[i,i])
    print('The 95% CI for beta{} is: [{},{}]'.format(i,left,right))

The 95% CI for beta0 is: [0.6458119644840438,1.7906903025172558]
The 95% CI for beta1 is: [0.49709537347828703,1.8492722772197514]
The 95% CI for beta2 is: [0.8827919869847044,2.2458025837571167]
The 95% CI for beta3 is: [-0.8531103752581496,0.5435568165577712]


In [3]:
from scipy.stats import f
SSE = np.dot((y-y_hat).transpose(),(y-y_hat))
SSR = np.dot((y_hat-np.mean(y)).transpose(),(y_hat-np.mean(y)))
F = (SSR/3)/(SSE/96)
p = 1 - f.cdf(F, 3, 96)
print('The SSE is',SSE)
print('The SSR is',SSR)
print('The value for F statistic is',F)
print('The corresponding p-value is',p)

The SSE is 107.57028768083038
The SSR is 32.51294472333451
The value for F statistic is 9.671948021871025
The corresponding p-value is 1.2245228216856141e-05


In [4]:
count = 0
for i in range(1000):
    u = np.random.normal(0, 1, 100)
    X = np.random.uniform(0, 1, (100, 3))
    one = np.ones(100)
    beta = np.array([1,2,0])
    y = one + np.dot(X,beta) + u
    X = sm.add_constant(X)
    model = sm.OLS(y,X)
    results = model.fit()
    beta_hat = results.params
    y_hat = np.dot(X, beta_hat)
    cov = np.linalg.inv(np.dot(X.transpose(),X))
    T = beta_hat[3]/np.sqrt(cov[3,3])
    p = 1 - t.cdf(abs(T),96)+t.cdf(-abs(T),96)
    if p < 0.05:
        count+=1
print('''The number of simulations with p-values < 0.05 is around {}, which is close to 0.05*1000.
The empirical type I error rate is {}/1000 = {}, which is close to the significance level of 0.05.'''.format(count,count,count/1000))

The number of simulations with p-values < 0.05 is around 52, which is close to 0.05*1000.
The empirical type I error rate is 52/1000 = 0.052, which is close to the significance level of 0.05.


In [5]:
x0=np.array([1,0.3,0.2,0.7])
y0_hat = np.dot(x0, beta_hat)
S_square = SSE/96
cov = S_square*np.dot(np.dot(x0,np.linalg.inv(np.dot(X.transpose(),X))),x0.transpose())
left = y0_hat - np.sqrt(cov)*t.ppf(0.975, 96)
right = y0_hat + np.sqrt(cov)*t.ppf(0.975, 96)
print('The predicted value for y0 is',y0_hat)
print('The prediction interval for y0 is [{},{}]'.format(left,right))

The predicted value for y0 is 1.515431050232146
The prediction interval for y0 is [1.1701203367307076,1.8607417637335844]


In [6]:
n_list = [200,500,1000]
wide_list = []
for n in n_list:
    u = np.random.normal(0, 1, n)
    X = np.random.uniform(0, 1, (n, 3))
    one = np.ones(n)
    beta = np.array([1,2,0])
    y = one + np.dot(X,beta) + u
    X = sm.add_constant(X)
    model = sm.OLS(y,X)
    results = model.fit()
    beta_hat = results.params
    y_hat = np.dot(X, beta_hat)
    SSE = np.dot((y-y_hat).transpose(),(y-y_hat))
    x0=np.array([1,0.3,0.2,0.7])
    y0_hat = np.dot(x0, beta_hat)
    S_square = SSE/(n-4)
    cov =1 + S_square*np.dot(np.dot(x0,np.linalg.inv(np.dot(X.transpose(),X))),x0.transpose())
    wide_list.append(2*np.sqrt(cov)*t.ppf(0.975, n-4))
print('''The width of 95% prediction intervals for y0 are: {}, {} and {} respectively for n = 200, 500 and 1000. 
The width does not approach to 0, but gradually be close to its limit of 3.969969.'''.format(wide_list[0],wide_list[1],wide_list[2]))

The width of 95% prediction intervals for y0 are: 3.9727755116462533, 3.9417595695711127 and 3.930276137372049 respectively for n = 200, 500 and 1000. 
The width does not approach to 0, but gradually be close to its limit of 3.969969.


In [64]:
import pandas as pd
from pandas.core.frame import DataFrame
import statsmodels.api as sm
from statsmodels.formula.api import ols
data = pd.read_csv(r"C:\Users\86180\Downloads\traffic.csv")
list_clouds = []
list_clear = []
list_snow = []
for i in range(len(data['weather_main'])):
    if data['weather_main'][i] == 'Clouds':
        list_clouds.append(data['traffic_volume'][i])
    elif data['weather_main'][i] == 'Clear':
        list_clear.append(data['traffic_volume'][i])
    elif data['weather_main'][i] == 'Snow':
        list_snow.append(data['traffic_volume'][i])
class_list = [0 for _ in range(len(list_clouds))] + [1 for _ in range(len(list_clear))]+ [2 for _ in range(len(list_snow))]
print(len(class_list))
traffic_list = list_clouds + list_clear + list_snow
c={"classes" : class_list,"traffic" : traffic_list}
data1=DataFrame(c)
model = ols('traffic ~ C(classes)', data=data1).fit()
anovat = sm.stats.anova_lm(model)
print(anovat)

31431
                 df        sum_sq       mean_sq           F         PR(>F)
C(classes)      2.0  2.548512e+09  1.274256e+09  338.384375  3.994626e-146
Residual    31428.0  1.183486e+11  3.765705e+06         NaN            NaN


In [66]:
print(f.ppf(0.95, 2.0, 31428.0))

2.9960178463550626
